In [ ]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 71.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
from itertools import chain, combinations
import random

# Load your data into a pandas DataFrame


# Load the dataset
file_path = './Dataset 1.xlsx'
data = pd.read_excel(file_path)

# Remove unnecessary columns
data = data.drop(columns=['İLÇE ADI'])

# Convert the data to a NumPy array
dat1 = data.to_numpy()

# Replace diagonal with a very big number to avoid self-loops
np.fill_diagonal(dat1, 10000000)

# Set a random seed for reproducibility
random.seed(42)

num_districts = data.shape[0]

# Define the powerset function
def powerset(iterable):
    "powerset([1,2,3]) --> [()] [1] [2] [3] [1,2] [1,3] [2,3] [1,2,3]"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

# Define the callback function for lazy constraints
def subtour_elimination(model, where):
    if where == GRB.Callback.MIPSOL:
        # Get the current solution
        vals = model.cbGetSolution(model._x)
        # Create a list of selected edges based on the current solution
        selected = gp.tuplelist((i, j) for i in range(num_districts) for j in range(num_districts) if vals[i, j] > 0.5)

        # Find the shortest subtour in the selected edges
        tour = find_subtour(selected)
        if len(tour) < num_districts:
            # If a subtour is found, add a lazy constraint to eliminate it
            model.cbLazy(gp.quicksum(model._x[i, j] for i in tour for j in tour if i != j) <= len(tour) - 1)

# Function to find the shortest subtour
def find_subtour(edges):
    unvisited = list(range(num_districts))
    cycle = range(num_districts + 1)
    while unvisited:
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*') if j in unvisited]
        if len(thiscycle) < len(cycle):
            cycle = thiscycle
    return cycle

# Create a new Gurobi model
m = gp.Model("TSP_HW3_All")

# Enable detailed logging
m.setParam('OutputFlag', 1)
m.setParam('MIPGap', 1e-4)  # Adjust the optimality gap threshold as needed

# Define decision variables
x = m.addVars(num_districts, num_districts, vtype=GRB.BINARY, name="x")

# Set objective function
m.setObjective(gp.quicksum(dat1[i, j] * x[i, j] for i in range(num_districts) for j in range(num_districts)), GRB.MINIMIZE)

# Constraints to ensure each node is visited exactly once
for i in range(num_districts):
    m.addConstr(gp.quicksum(x[i, j] for j in range(num_districts) if i != j) == 1, f"row_{i}")

for j in range(num_districts):
    m.addConstr(gp.quicksum(x[i, j] for i in range(num_districts) if i != j) == 1, f"col_{j}")

# Set the model to use the lazy constraints callback
m._x = x
m.Params.LazyConstraints = 1

# Optimize the model using the subtour elimination callback
m.optimize(subtour_elimination)

# Print the optimal cost if an optimal solution is found
if m.status == GRB.OPTIMAL:
    print(f"Total Optimal Cost = {m.objVal}")

    # Print the solution
    solution = m.getAttr('x', x)
    for i in range(num_districts):
        for j in range(num_districts):
            # Gurobi uses a tolerance, so we check for values greater than 0.5
            if solution[i, j] > 0.5:
                print(f"X[{i},{j}] = 1")


Restricted license - for non-production use only - expires 2025-11-24
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 76 rows, 1444 columns and 2812 nonzeros
Model fingerprint: 0xcdea7dd8
Variable types: 0 continuous, 1444 integer (1444 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 1e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 1357.1141531
Presolve time: 0.03s
Presolved: 76 rows, 1444 columns, 2812 nonzeros
Variable types: 0 continuous, 1444 integer (1444 binary)

Root relaxation: objective 4.303450e+02, 66 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Un

In [ ]:
data.columns

Index(['ARNAVUTKÖY', 'ATAŞEHİR', 'AVCILAR', 'BAĞCILAR', 'BAHÇELİEVLER',
       'BAKIRKÖY', 'BAŞAKŞEHİR', 'BAYRAMPAŞA', 'BEŞİKTAŞ', 'BEYKOZ',
       'BEYLİKDÜZÜ', 'BEYOĞLU', 'BÜYÜKÇEKMECE', 'ÇATALCA', 'ÇEKMEKÖY',
       'ESENLER', 'ESENYURT', 'EYÜP', 'FATİH', 'GAZİOSMANPAŞA', 'GÜNGÖREN',
       'KADIKÖY', 'KAĞITHANE', 'KARTAL', 'KÜÇÜKÇEKMECE', 'MALTEPE', 'PENDİK',
       'SANCAKTEPE', 'SARIYER', 'SİLİVRİ', 'SULTANBEYLİ', 'SULTANGAZİ', 'ŞİLE',
       'ŞİŞLİ', 'TUZLA', 'ÜMRANİYE', 'ÜSKÜDAR', 'ZEYTİNBURNU'],
      dtype='object')

In [ ]:
tour = [ (0,13),(1,21),(2,24),(3,6),(4,5),(5,20),(6,31),(7,18),(8,33),(9,14),(10,16),(11,0),(12,10),(13,29),(14,27),(15,3),(16,2),(17,15),(18,17),(19,22),(20,37),(21,36),(22,28),(23,25),(24,4),(25,1),(26,30),(27,32),(28,35),(29,12),(30,23),(31,19),(32,34),(33,11),(34,26),(35,9),(36,8),(37,7)]

In [ ]:
district_names = [
 'ARNAVUTKÖY, Istanbul', 'ATAŞEHİR, Istanbul', 'AVCILAR, Istanbul', 'BAĞCILAR, Istanbul', 'BAHÇELİEVLER, Istanbul',
       'BAKIRKÖY, Istanbul', 'BAŞAKŞEHİR, Istanbul', 'BAYRAMPAŞA, Istanbul', 'BEŞİKTAŞ, Istanbul', 'BEYKOZ, Istanbul',
       'BEYLİKDÜZÜ, Istanbul', 'BEYOĞLU, Istanbul', 'BÜYÜKÇEKMECE, Istanbul', 'ÇATALCA, Istanbul', 'ÇEKMEKÖY, Istanbul',
       'ESENLER, Istanbul', 'ESENYURT, Istanbul', 'EYÜP, Istanbul', 'FATİH, Istanbul', 'GAZİOSMANPAŞA, Istanbul', 'GÜNGÖREN, Istanbul',
       'KADIKÖY, Istanbul', 'KAĞITHANE, Istanbul', 'KARTAL, Istanbul', 'KÜÇÜKÇEKMECE, Istanbul', 'MALTEPE, Istanbul', 'PENDİK, Istanbul',
       'SANCAKTEPE, Istanbul', 'SARIYER, Istanbul', 'SİLİVRİ, Istanbul', 'SULTANBEYLİ, Istanbul', 'SULTANGAZİ, Istanbul', 'ŞİLE, Istanbul',
       'ŞİŞLİ, Istanbul', 'TUZLA, Istanbul', 'ÜMRANİYE, Istanbul', 'ÜSKÜDAR, Istanbul', 'ZEYTİNBURNU'
]

def get_tsp_route(tour):
    route = []
    current_city = 0  # Starting from the first city
    route.append(current_city)
    route_names=[district_names[current_city]]
    while len(route) < num_districts:
        for i, j in tour:
            if i == current_city and j not in route:
                route.append(j)
                current_city = j
                route_names.append(district_names[current_city])
                break
    return route_names

get_tsp_route(tour)


['ARNAVUTKÖY, Istanbul',
 'ÇATALCA, Istanbul',
 'SİLİVRİ, Istanbul',
 'BÜYÜKÇEKMECE, Istanbul',
 'BEYLİKDÜZÜ, Istanbul',
 'ESENYURT, Istanbul',
 'AVCILAR, Istanbul',
 'KÜÇÜKÇEKMECE, Istanbul',
 'BAHÇELİEVLER, Istanbul',
 'BAKIRKÖY, Istanbul',
 'GÜNGÖREN, Istanbul',
 'ZEYTİNBURNU',
 'BAYRAMPAŞA, Istanbul',
 'FATİH, Istanbul',
 'EYÜP, Istanbul',
 'ESENLER, Istanbul',
 'BAĞCILAR, Istanbul',
 'BAŞAKŞEHİR, Istanbul',
 'SULTANGAZİ, Istanbul',
 'GAZİOSMANPAŞA, Istanbul',
 'KAĞITHANE, Istanbul',
 'SARIYER, Istanbul',
 'ÜMRANİYE, Istanbul',
 'BEYKOZ, Istanbul',
 'ÇEKMEKÖY, Istanbul',
 'SANCAKTEPE, Istanbul',
 'ŞİLE, Istanbul',
 'TUZLA, Istanbul',
 'PENDİK, Istanbul',
 'SULTANBEYLİ, Istanbul',
 'KARTAL, Istanbul',
 'MALTEPE, Istanbul',
 'ATAŞEHİR, Istanbul',
 'KADIKÖY, Istanbul',
 'ÜSKÜDAR, Istanbul',
 'BEŞİKTAŞ, Istanbul',
 'ŞİŞLİ, Istanbul',
 'BEYOĞLU, Istanbul']

In [ ]:

import folium
from geopy.geocoders import Nominatim
# List of district names


# Get coordinates of the districts using Geopy
geolocator = Nominatim(user_agent="tsp_map")
locations = {}
for district in district_names:
    location = geolocator.geocode(district + ", Turkey")
    if location:
        locations[district] = (location.latitude, location.longitude)
    else:
        print(f"Could not find coordinates for {district}")

# Create a Folium map centered around Turkey
mymap = folium.Map(location=[39.0, 35.0], zoom_start=6)

# Add district markers to the map
for district, coord in locations.items():
    folium.Marker(location=coord, popup=district).add_to(mymap)

# Draw the TSP path
for i, j in tour:
    district_i = district_names[i]
    district_j = district_names[j]
    if district_i in locations and district_j in locations:
        coord_i = locations[district_i]
        coord_j = locations[district_j]
        folium.PolyLine([coord_i, coord_j], color="blue", weight=2.5, opacity=1).add_to(mymap)

# Save the map to an HTML file
mymap.save("tsp_turkey_map.html")

# Display the map (optional, in Jupyter notebook or similar environments)
mymap